# ispy-jupyter

A demonstrator in which we read in NANOAOD and render it using three.js (via pythreejs) in a jupyter notebook

1. Read in nanoAOD and select
2. Create a renderer, scene, camera, etc.
3. Render an event

In [1]:
import math
import uproot
import os

import awkward as ak
import ipywidgets as widgets
import numpy as np

from pythreejs import *
from IPython.display import display

## Part 1: nanoAOD

[Simulated dataset QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8 in NANOAODSIM format for 2016 collision data](https://opendata.cern.ch/record/63168)

In [2]:
infile_name = '397D1673-167A-CF46-9E79-D7069D9AC359.root'

if not (os.path.isfile(infile_name)): 
    !curl -O http://opendata.cern.ch/eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/270000/397D1673-167A-CF46-9E79-D7069D9AC359.root

In [3]:
infile = uproot.open(infile_name)

In [4]:
events = infile['Events']
events.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
run                  | uint32_t                 | AsDtype('>u4')
luminosityBlock      | uint32_t                 | AsDtype('>u4')
event                | uint64_t                 | AsDtype('>u8')
HTXS_Higgs_pt        | float                    | AsDtype('>f4')
HTXS_Higgs_y         | float                    | AsDtype('>f4')
HTXS_stage1_1_cat... | int32_t                  | AsDtype('>i4')
HTXS_stage1_1_cat... | int32_t                  | AsDtype('>i4')
HTXS_stage1_1_fin... | int32_t                  | AsDtype('>i4')
HTXS_stage1_1_fin... | int32_t                  | AsDtype('>i4')
HTXS_stage1_2_cat... | int32_t                  | AsDtype('>i4')
HTXS_stage1_2_cat... | int32_t                  | AsDtype('>i4')
HTXS_stage1_2_fin... | int32_t                  | AsDtype('>i4')
HTXS_stage1_2_fin... | int32_t                  | AsDtype(

[Dataset semantics](https://opendata.cern.ch/eos/opendata/cms/dataset-semantics/NanoAODSIM/63168/QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8_doc.html)

## Part 2: three.js

Create a scene, renderer, controls, and lights

In [5]:
lp = 15.0

lights = [
    DirectionalLight(color='white', position=[-lp, lp, lp], intensity=1),
    DirectionalLight(color='white', position=[lp, -lp, -lp], intensity=1)
]

In [6]:
width = 800
height = 500

camera = PerspectiveCamera(
    position=[5, 5, 10],
    up=[0, 1, 0], 
    children=[lights],
    aspect=width/height
)

In [7]:
scene = Scene(
    background='#232323'
)

In [8]:
renderer = Renderer(
    camera=camera,
    scene=scene,
    controls=[OrbitControls(controlling=camera)],
    width=width,
    height=height
)

In [9]:
def make_jet(pt, eta, phi):

    theta = 2*math.atan(pow(math.e, -eta))

    dir = np.array([
        math.cos(phi),
        math.sin(phi),
        math.sinh(eta)
    ])

    dir /= np.linalg.norm(dir)
    
    radius = 0.3*(1.0/(1+0.001))
    length = 1.10
    
    geometry = CylinderGeometry(
        radiusTop=radius,
        radiusBottom=0.0,
        height=length,
        radialSegments=32,
        heightSegments=1,
        openEnded=True
    )

    length *= 0.5
    
    jet = Mesh(
        geometry=geometry,
        material=MeshBasicMaterial(
            color='#ffff00', 
            side='DoubleSide',
            transparent=True,
            opacity=0.5
        )
    )

    jet.rotateZ(phi-math.pi/2)
    jet.rotateX(math.pi/2-theta)
    jet.position = (dir*length).tolist()

    return jet

make_jets = np.vectorize(make_jet)

In [11]:
def make_arrow(eta, phi):
 
    dir = np.array([
        math.cos(phi),
        math.sin(phi),
        math.sinh(eta)
    ])

    dir /= np.linalg.norm(dir)
    
    arrow = ArrowHelper(
      dir=dir.tolist(),
      origin=[0, 0, 0],
      length=2,
      color='#00ff00',
      headLength=0.2,
      headWidth=0.2
    )

    return arrow

For now just draw electrons and muons as arrows

In [12]:
def make_muon(pt, eta, phi, charge):

    dir = np.array([
        math.cos(phi),
        math.sin(phi),
        math.sinh(eta)
    ])

    dir /= np.linalg.norm(dir)
    
    muon = ArrowHelper(
      dir=dir.tolist(),
      origin=[0, 0, 0],
      length=pt*0.1,
      color='#ff0000',
      headLength=0.2,
      headWidth=0.1
    )

    return muon

make_muons = np.vectorize(make_muon)

In [13]:
def make_electron(pt, eta, phi, charge):

    dir = np.array([
        math.cos(phi),
        math.sin(phi),
        math.sinh(eta)
    ])

    dir /= np.linalg.norm(dir)
    
    electron = ArrowHelper(
      dir=dir.tolist(),
      origin=[0, 0, 0],
      length=pt*0.1,
      color='#19ff19',
      headLength=0.2,
      headWidth=0.1
    )

    return electron

make_electrons = np.vectorize(make_electron)

In [14]:
def make_met(pt, phi):

    px = math.cos(phi)
    py = math.sin(phi)
    
    dir = np.array([px,py,0])
    dir /= np.linalg.norm(dir)
    d = 1.24
    length = pt*0.1
    length = 5 if length+d > 5 else length
    
    met = ArrowHelper(
      dir=dir.tolist(),
      origin=[px*d, py*d, 0],
      length=length,
      color='#ff00ff',
      headLength=0.2,
      headWidth=0.2
    )

    return met

In [15]:
def make_sv(x,y,z):

    geometry = SphereGeometry(
        radius=0.01,
        widthSegments=32,
        heightSegments=32
    )

    vertex = Mesh(
        geometry,
        MeshBasicMaterial(color='#ff6600')
    )
    
    vertex.position = [0.01*x,0.01*y,0.01*z]
    
    return vertex

make_svs = np.vectorize(make_sv)

def make_pv(position):

    geometry = SphereGeometry(
        radius=0.01,
        widthSegments=32,
        heightSegments=32
    )

    vertex = Mesh(
        geometry,
        MeshBasicMaterial(color='#ffff00')
    )

    vertex.position = position

    return vertex

Make an EB-like geometry for context

In [16]:
def make_eb():

    geometry = CylinderGeometry(
        radiusTop=1.12, 
        radiusBottom=1.24, 
        height=6.0, 
        radialSegments=64, 
        heightSegments=1, 
        openEnded=True, 
        thetaStart=0, 
        thetaLength=2*math.pi
    )

    eb = Mesh(
        geometry,
        MeshBasicMaterial(
            color='#7fccff',
            wireframe=True,
            transparent=True,
            opacity=0.2
        )
    )

    eb.rotateX(math.pi/2)
    
    return eb

## Part 3: Render events

In [17]:
df = events.arrays(
    [
        'nJet', 'Jet_pt', 'Jet_eta', 'Jet_phi',
        'MET_pt', 'MET_phi',
        'nPhoton', 'Photon_pt', 'Photon_eta', 'Photon_phi',
        'nMuon', 'Muon_pt', 'Muon_eta', 'Muon_phi', 'Muon_charge',
        'nElectron', 'Electron_pt', 'Electron_eta', 'Electron_phi', 'Electron_charge',
        'nSV', 'SV_x', 'SV_y', 'SV_z',
        'PV_x', 'PV_y', 'PV_z'
    ],
    library='ak'
)

In [18]:
df

<Array [{nJet: 2, Jet_pt: [...], ...}, ...] type='2000 * {nJet: uint32, Jet...'>

In [19]:
def add_event(event):

    scene.children = []
    
    met = make_met(
        event['MET_pt'],
        event['MET_phi']
    )

    scene.add(met)

    if event['nJet'] > 0:
    
        jets = make_jets(
            event['Jet_pt'],
            event['Jet_eta'],
            event['Jet_phi']
        )

        scene.add(jets)

    if event['nElectron'] > 0:

        electrons = make_electrons(
            event['Electron_pt'],
            event['Electron_eta'],
            event['Electron_phi'],
            event['Electron_charge']
        )

        scene.add(electrons)

    if event['nMuon'] > 0:
        
        muons = make_muons(
            event['Muon_pt'],
            event['Muon_eta'],
            event['Muon_phi'],
            event['Muon_charge']
        )

        scene.add(muons)
        
    if event['nSV'] > 0:
        
        svs = make_svs(
            event['SV_x'],
            event['SV_y'],
            event['SV_z']
        )
         
        scene.add(svs)

    pv = make_pv([
        0.01*event['PV_x'],
        0.01*event['PV_y'],
        0.01*event['PV_z']
    ])

    scene.add(pv)
    
    scene.add(make_eb())

In [20]:
renderer.render(scene, camera)

current_event = 0
max_events = len(df)-1

prev_button = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Previous Event',
    icon='step-backward'
)

next_button = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Next Event',
    icon='step-forward'
)

output = widgets.Output()

def prev_button_clicked(b):
    with output:
        global current_event
        if current_event > 0:
            current_event -= 1
            add_event(df[current_event])
        print('event', current_event)

def next_button_clicked(b):
    with output:
        global current_event
        if current_event < max_events:
            current_event += 1
            add_event(df[current_event])
        print('event', current_event)

prev_button.on_click(prev_button_clicked)
next_button.on_click(next_button_clicked)

In [21]:
add_event(df[current_event])

In [22]:
renderer

Renderer(camera=PerspectiveCamera(aspect=1.6, children=([DirectionalLight(color='white', matrixWorldNeedsUpdat…

In [24]:
display(widgets.HBox((prev_button, next_button)), output)

Output()